In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [34]:
train = pd.read_csv('/content/drive/MyDrive/train.csv', index_col=[0])
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [36]:
train.shape

(11504798, 11)

In [37]:
test.shape

(7669866, 11)

In [38]:
def label_encode_gender(val):
    if val == 'Female':
        return 1
    elif val == 'Male':
        return 0
    return None

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
train['Gender'] = train['Gender'].apply(label_encode_gender)
test['Gender'] = test['Gender'].apply(label_encode_gender)

In [41]:
def label_encode_vehicle_age(val):
    if val == '< 1 Year':
        return 0
    elif val == '1-2 Year':
        return 1
    elif val == '> 2 Years':
        return 2
    return None

In [42]:
train['Vehicle_Age'] = train['Vehicle_Age'].apply(label_encode_vehicle_age)
test['Vehicle_Age'] = test['Vehicle_Age'].apply(label_encode_vehicle_age)

In [43]:
def label_encode_vehicle_damage(val):
    if val == 'Yes':
        return 1
    else:
        return 0
    return None

In [44]:
train['Vehicle_Damage'] = train['Vehicle_Damage'].apply(label_encode_vehicle_damage)
test['Vehicle_Damage'] = test['Vehicle_Damage'].apply(label_encode_vehicle_damage)

In [45]:
y = train['Response']
X = train.drop(columns=['Response'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [46]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [47]:
y_pred_train_proba = model.predict_proba(X_train)[:, 1]
y_pred_test_proba = model.predict_proba(X_test)[:, 1]

print(f'Train AUC = {roc_auc_score(y_train, y_pred_train_proba)}')
print(f'Test AUC = {roc_auc_score(y_test, y_pred_test_proba)}')

Train AUC = 0.8792429062145518
Test AUC = 0.8780688645398665


In [48]:
final_model = XGBClassifier()
final_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [49]:
response_values = final_model.predict_proba(test.iloc[:, 1:])[:, 1]

In [50]:
response_values

array([7.6249400e-03, 4.4855863e-01, 2.4091090e-01, ..., 2.5187770e-04,
       5.6185889e-01, 9.3597708e-05], dtype=float32)

In [51]:
test['id'].values

array([11504798, 11504799, 11504800, ..., 19174661, 19174662, 19174663])

In [52]:
data = {
    'id': test['id'].values,
    'Response': response_values
}

df = pd.DataFrame(data)

df.to_csv('submission.csv', index=False)

In [53]:
df

,id,Response
0,11504798,0.007625
1,11504799,0.448559
2,11504800,0.240911
3,11504801,0.000103
4,11504802,0.050063
...,...,...
7669861,19174659,0.208461
7669862,19174660,0.000135
7669863,19174661,0.000252
7669864,19174662,0.561859


In [54]:
df.shape

(7669866, 2)